# MLB Playoff Predictions
We can take historical MLB team statistics to train a model that can (inacurrately) predict the world series champions.

In [ ]:
import bigframes.pandas as bpd

In [ ]:
# ! gcloud auth login
# ! gcloud config set project jessekleve
# %set_env GOOGLE_APPLICATION_CREDENTIALS=/Users/jesse/.config/gcloud/application_default_credentials.json
%set_env GOOGLE_APPLICATION_CREDENTIALS=/Users/jesse/jessekleve.key.json

bpd.options.display.max_rows = 100
bpd.options.bigquery.location = "US"

# Load some data from big query

This will do stuff behind the scenes but is not pulling records from Big Query locally. It creates a session and a temp table.

In [66]:
team_data = bpd.read_gbq("jessekleve.baseball.teams")
champions = bpd.read_gbq("jessekleve.baseball.world_series")

HTML(value='Query job 4b6cd88a-2a5c-4fa9-85c4-f28ffdad5ec5 is RUNNING. <a target="_blank" href="https://consol…

HTML(value='Query job 338979ae-6663-415c-ae15-53b7def11981 is RUNNING. <a target="_blank" href="https://consol…

### Metadata
We can look at some metadata about the table like the column names and types.

In [68]:
team_data.dtypes

yearID                      Int64
lgID              string[pyarrow]
teamID            string[pyarrow]
franchID          string[pyarrow]
divID             string[pyarrow]
Rank                        Int64
G                           Int64
Ghome                       Int64
W                           Int64
L                           Int64
DivWin                    boolean
WCWin                     boolean
LgWin                     boolean
WSWin                     boolean
R                           Int64
AB                          Int64
H                           Int64
_2B                         Int64
_3B                         Int64
HR                          Int64
BB                          Int64
SO                          Int64
SB                          Int64
CS                          Int64
HBP                         Int64
SF                          Int64
RA                          Int64
ER                          Int64
ERA                       Float64
CG            

In [69]:
champions.dtypes

yearID                    Int64
round           string[pyarrow]
teamIDwinner    string[pyarrow]
lgIDwinner      string[pyarrow]
teamIDloser     string[pyarrow]
lgIDloser       string[pyarrow]
wins                      Int64
losses                    Int64
ties                      Int64
dtype: object

In [70]:
team_data.sort_values(by="yearID").tail(5)

HTML(value='Query job bc5d88e7-23f3-42bc-af4a-e92433133426 is DONE. 44.9 kB processed. <a target="_blank" href…

HTML(value='Query job 957d4d77-d9d7-4574-8ab5-aa9e0cc78fcd is DONE. 947.6 kB processed. <a target="_blank" hre…

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
2401,2015,NL,PIT,PIT,C,2,162,81,98,64,...,177,0.981,Pittsburgh Pirates,PNC Park,2498596,99,97,PIT,PIT,PIT
2458,2015,NL,SDN,SDP,W,4,162,81,74,88,...,138,0.985,San Diego Padres,Petco Park,2459742,98,97,SDP,SDN,SDN
2591,2015,NL,SFN,SFG,W,2,162,81,84,78,...,145,0.987,San Francisco Giants,AT&T Park,3375882,99,97,SFG,SFN,SFN
2717,2015,NL,SLN,STL,C,1,162,81,100,62,...,159,0.984,St. Louis Cardinals,Busch Stadium III,3520889,102,101,STL,SLN,SLN
2784,2015,NL,WAS,WSN,E,2,162,81,83,79,...,125,0.985,Washington Nationals,Nationals Park,2619843,102,99,WSN,MON,WAS


In [71]:
champions = champions[champions["yearID"] >= 2015]

In [72]:
train_data = champions.merge(team_data, how="inner", on="yearID")

In [73]:
train_data.shape

HTML(value='Query job f048bd2a-5487-4cbd-ad82-b0b34336e52b is DONE. 24.9 kB processed. <a target="_blank" href…

(270, 56)

# Assign the label & prep the training data
Here we can create a new row in the data which is a binary variable on whether the team won the world series or not. This will be the label that we train our model with. It's the output binary variable.

In [75]:
train_data = train_data.assign(won=train_data["teamID"] == train_data["teamIDwinner"])

In [76]:
# train_data.sort_values(by="won")

In [77]:
# train_data[["teamID", "teamIDwinner", "yearID", "won"]].head(1000)

In [78]:
# fields = ["Rank", "W", "HR", "SO", "SB", "ER", "ERA", "FP"]
fields = ["Rank", "W", "HR", "ER", "ERA", "FP"]
# fields = ["W", "ER", "ERA", "FP"]


X = train_data[fields]
y = train_data[["won"]]

In [79]:
# X.FP.isnull().unique()

In [80]:
# X = X.dropna()

In [81]:
# X.shape

# Create a ML model

Now with our training data we can submit jobs to Big Query that'll train a logistic regression model on our training data we have selected.

In [82]:
import bigframes.ml.linear_model as ml
model = ml.LogisticRegression()

#### Fit the model using our training data

In [83]:
model.fit(X, y)

HTML(value='Query job f91d347a-5911-4e16-881b-0841ebce7a6f is RUNNING. <a target="_blank" href="https://consol…

LogisticRegression()

# Save model

With the model trained, we can save it for future use or even publish it to make it accessible to our applications through Google's API's. Training a model might take quite a bit of time and a lot of compute resources. Saving it and reloading it will save us time and money.

In [ ]:
model.to_gbq("jessekleve.baseball.world_series_model", replace=True)

#### If you wanted to load the model

In [ ]:
# model = bpd.read_gbq_model("jessekleve.baseball.world_series_model")

# Predict the world series!

In [84]:
data_2023 = bpd.read_gbq("jessekleve.baseball.data_2023")

HTML(value='Query job a39de333-c2b0-4401-98f5-36d176c96a2e is RUNNING. <a target="_blank" href="https://consol…

In [85]:
data_2023

HTML(value='Query job 6c607dde-ff0c-4b9b-b056-6b462e8e0ea8 is DONE. 0 Bytes processed. <a target="_blank" href…

HTML(value='Query job 65fd1721-1dff-4ce9-8876-332930e903b3 is DONE. 1.2 kB processed. <a target="_blank" href=…

,name,yearID,teamID,Rank,W,HR,SO,SB,ER,ERA,FP
0,Atlanta Braves,2023,ATL,1,104,187,1516,132,662,108,0.986
1,Baltimore Orioles,2023,BAL,1,101,177,1431,114,629,106,0.988
2,Los Angeles Dodgers,2023,LAN,1,100,200,1388,105,652,108,0.987
3,Milwaukee Brewers,2023,ML4,1,92,198,1425,129,595,116,0.987
4,Minnesota Twins,2023,MIN,1,87,194,1560,86,624,111,0.988
5,Texas Rangers,2023,TEX,1,90,198,1351,79,683,105,0.99
6,Arizona Diamondbacks,2023,ARI,2,84,197,1351,166,713,98,0.99
7,Houston Astros,2023,HOU,2,90,201,1460,107,632,107,0.986
8,Philadelphia Phillies,2023,PHI,2,90,185,1454,141,645,107,0.984
9,Tampa Bay Rays,2023,TBA,2,99,177,1507,160,618,108,0.987


In [86]:
model.predict(data_2023[fields])

HTML(value='Query job efadca47-cd61-4ff9-aae3-f95caad22dab is RUNNING. <a target="_blank" href="https://consol…

HTML(value='Query job 36cacb28-3298-444e-ab92-bc7976526b72 is DONE. 96 Bytes processed. <a target="_blank" hre…

HTML(value='Query job 58902e59-42cd-4d25-b7e7-b411304b0822 is DONE. 0 Bytes processed. <a target="_blank" href…

HTML(value='Query job 16ab0825-22b1-46de-919a-751615d8e3fe is DONE. 108 Bytes processed. <a target="_blank" hr…

,predicted_won
0,False
1,False
2,False
3,False
4,False
5,False
6,False
7,False
8,False
9,False
